In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("../data/youtube_shorts_tiktok_trends_2025.csv_ML.csv")


In [13]:
import sys
sys.path.append("../src")

from preprocess import map_labels, clean_features, get_feature_matrix
from model_utils import rate_video


In [14]:
df_country = pd.read_csv("../data/country_platform_summary_2025.csv")
df_country.head()

df_country.rename(columns={'country': 'region'}, inplace=True)

df_enriched = df.merge(
    df_country[['region', 'platform', 'median_er', 'avg_velocity', 'avg_engagement_per_1k']],
    on=['region', 'platform'],
    how='left'
)
# Compute region+platform "strength" from your own ML data
region_platform_strength = (
    df.groupby(['region', 'platform'])
      .agg({
          'views_per_day': 'mean',
          'like_rate': 'mean',
          'share_rate': 'mean',
          'rel_like': 'mean',
          'rel_share': 'mean'
      })
      .reset_index()
      .rename(columns={
          'views_per_day': 'region_platform_avg_views_per_day',
          'like_rate': 'region_platform_avg_like_rate',
          'share_rate': 'region_platform_avg_share_rate',
          'rel_like': 'region_platform_avg_rel_like',
          'rel_share': 'region_platform_avg_rel_share'
      })
)

region_platform_strength.head()




,region,platform,region_platform_avg_views_per_day,region_platform_avg_like_rate,region_platform_avg_share_rate,region_platform_avg_rel_like,region_platform_avg_rel_share
0,Africa,tiktok,0.125424,0.237473,0.070539,0.172764,0.334977
1,Africa,youtube,0.103872,0.205414,0.059469,0.014196,0.125933
2,Americas,tiktok,0.125526,0.229555,0.071589,0.146339,0.338209
3,Americas,youtube,0.103101,0.203117,0.058582,0.015032,0.096152
4,Asia,tiktok,0.125880,0.236992,0.074647,0.153687,0.304208


In [ ]:
df = map_labels(df)
df['trend_bucket'].value_counts()




trend_bucket
likely      32402
trending    12500
low          5098
Name: count, dtype: int64

In [16]:
df_enriched = df.merge(
    region_platform_strength,
    on=['region', 'platform'],
    how='left'
)

df_enriched[['region', 'platform',
             'region_platform_avg_views_per_day',
             'region_platform_avg_like_rate']].head()

,region,platform,region_platform_avg_views_per_day,region_platform_avg_like_rate
0,Africa,youtube,0.103872,0.205414
1,Europe,youtube,0.103244,0.203507
2,Americas,tiktok,0.125526,0.229555
3,Americas,tiktok,0.125526,0.229555
4,Americas,tiktok,0.125526,0.229555


In [17]:
features = [
    'title_len', 'text_richness',
    'like_rate', 'comment_rate', 'share_rate',
    'views_per_day', 'likes_per_day',
    'rel_like', 'rel_share', 'rel_combo',
    'like_hashtag_interaction', 'share_hashtag_interaction',
    'platform_cat', 'region_cat', 'language_cat', 
    'category_cat', 'traffic_source_cat',
    'device_brand_cat', 'creator_tier_cat',
    'richness_traffic_interaction', 'weekend_hashtag_boost',
        'region_platform_avg_views_per_day',
    'region_platform_avg_like_rate',
    'region_platform_avg_share_rate',
    'region_platform_avg_rel_like',
    'region_platform_avg_rel_share'

]
df_clean = clean_features(df_enriched, features)
X, y = get_feature_matrix(df_clean, features)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [25]:
balanced_samples = (
    df_clean
    .groupby('trend_bucket', group_keys=False)
    .apply(lambda g: g.sample(min(3, len(g)), random_state=42))
)

balanced_samples[['trend_bucket']].head(10)


C:\Users\indyj\AppData\Local\Temp\ipykernel_10832\2949953786.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(min(3, len(g)), random_state=42))


,trend_bucket
39130,likely
17089,likely
7355,likely
9774,low
12771,low
8420,low
6959,trending
47591,trending
35851,trending


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

log_reg = LogisticRegression(max_iter=3000, multi_class='multinomial')
log_reg.fit(X_train, y_train)

pred_lr = log_reg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred_lr))
print("Macro F1:", f1_score(y_test, pred_lr, average='macro'))
print(confusion_matrix(y_test, pred_lr))
print(classification_report(y_test, pred_lr))


c:\Users\indyj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.8261
Macro F1: 0.759347284002688
[[5947  157  376]
 [ 434  572   14]
 [ 753    5 1742]]
              precision    recall  f1-score   support

      likely       0.83      0.92      0.87      6480
         low       0.78      0.56      0.65      1020
    trending       0.82      0.70      0.75      2500

    accuracy                           0.83     10000
   macro avg       0.81      0.73      0.76     10000
weighted avg       0.82      0.83      0.82     10000



In [19]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=400, random_state=42)
rf.fit(X_train, y_train)

pred_rf = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred_rf))
print("Macro F1:", f1_score(y_test, pred_rf, average='macro'))
print(confusion_matrix(y_test, pred_rf))
print(classification_report(y_test, pred_rf))


Accuracy: 0.8379
Macro F1: 0.7974843250357413
[[5904  196  380]
 [ 237  768   15]
 [ 781   12 1707]]
              precision    recall  f1-score   support

      likely       0.85      0.91      0.88      6480
         low       0.79      0.75      0.77      1020
    trending       0.81      0.68      0.74      2500

    accuracy                           0.84     10000
   macro avg       0.82      0.78      0.80     10000
weighted avg       0.84      0.84      0.83     10000



In [26]:
test_videos = [
    row[features].to_dict()
    for _, row in balanced_samples.iterrows()
]


for _, row in sample_rows.iterrows():
    test_videos.append(row[features].to_dict())

for i, video in enumerate(test_videos):
    print(f"\n--- Test Video {i+1} ---")
    display(pd.DataFrame(video, index=[0]))



--- Test Video 1 ---


,title_len,text_richness,like_rate,comment_rate,share_rate,views_per_day,likes_per_day,rel_like,rel_share,rel_combo,...,traffic_source_cat,device_brand_cat,creator_tier_cat,richness_traffic_interaction,weekend_hashtag_boost,region_platform_avg_views_per_day,region_platform_avg_like_rate,region_platform_avg_share_rate,region_platform_avg_rel_like,region_platform_avg_rel_share
0,2,1.999998,0.31263,0.108264,0.071943,0.099262,0.031032,0.543434,0.319764,0.117476,...,4,0,3,9.99999,0,0.12588,0.236992,0.074647,0.153687,0.304208



--- Test Video 2 ---


,title_len,text_richness,like_rate,comment_rate,share_rate,views_per_day,likes_per_day,rel_like,rel_share,rel_combo,...,traffic_source_cat,device_brand_cat,creator_tier_cat,richness_traffic_interaction,weekend_hashtag_boost,region_platform_avg_views_per_day,region_platform_avg_like_rate,region_platform_avg_share_rate,region_platform_avg_rel_like,region_platform_avg_rel_share
0,2,1.999998,0.095362,0.051064,0.063192,0.115633,0.011027,-0.530667,0.26782,-0.118992,...,1,5,2,3.999996,0,0.103244,0.203507,0.059564,0.004827,0.106287



--- Test Video 3 ---


...

,title_len,text_richness,like_rate,comment_rate,share_rate,views_per_day,likes_per_day,rel_like,rel_share,rel_combo,...,traffic_source_cat,device_brand_cat,creator_tier_cat,richness_traffic_interaction,weekend_hashtag_boost,region_platform_avg_views_per_day,region_platform_avg_like_rate,region_platform_avg_share_rate,region_platform_avg_rel_like,region_platform_avg_rel_share
0,3,2.999997,0.334572,0.049647,0.060681,0.12032,0.040256,0.682384,0.169166,0.25556,...,1,6,3,5.999994,0,0.103101,0.203117,0.058582,0.015032,0.096152



--- Test Video 11 ---


,title_len,text_richness,like_rate,comment_rate,share_rate,views_per_day,likes_per_day,rel_like,rel_share,rel_combo,...,traffic_source_cat,device_brand_cat,creator_tier_cat,richness_traffic_interaction,weekend_hashtag_boost,region_platform_avg_views_per_day,region_platform_avg_like_rate,region_platform_avg_share_rate,region_platform_avg_rel_like,region_platform_avg_rel_share
0,2,1.999998,0.159517,0.110338,0.003468,0.085519,0.013642,-0.177843,-0.938238,-0.292968,...,1,6,2,3.999996,0,0.103274,0.209323,0.06291,0.018273,0.099448



--- Test Video 12 ---


,title_len,text_richness,like_rate,comment_rate,share_rate,views_per_day,likes_per_day,rel_like,rel_share,rel_combo,...,traffic_source_cat,device_brand_cat,creator_tier_cat,richness_traffic_interaction,weekend_hashtag_boost,region_platform_avg_views_per_day,region_platform_avg_like_rate,region_platform_avg_share_rate,region_platform_avg_rel_like,region_platform_avg_rel_share
0,2,1.999998,0.12236,0.006508,0.062449,0.072767,0.008904,-0.428182,0.046163,-0.334512,...,0,5,1,1.999998,0,0.103101,0.203117,0.058582,0.015032,0.096152


In [27]:


platform_map = df[['platform', 'platform_cat']].drop_duplicates().set_index('platform_cat')['platform'].to_dict()

region_map = df[['region', 'region_cat']].drop_duplicates().set_index('region_cat')['region'].to_dict()

for i, video in enumerate(test_videos):
    pred, score = rate_video(video, rf)
    print(f"Video {i+1}:")
    print(f"  Predicted Category: {pred}")
    print(f"  Trending Score: {score:.2f}/100")
    
    print(f"Platform: {platform_map[video['platform_cat']]}")
    print(f"Region: {region_map[video['region_cat']]}")

    
    if score > 60:
        print("  Interpretation: This video is highly likely to trend.")
    elif score > 20:
        print("  Interpretation: Decent performance but not strong enough to trend.")
    else:
        print("  Interpretation: Weak performance relative to region/platform norms.")
    
    print()







Video 1:
  Predicted Category: likely
  Trending Score: 6.25/100
Platform: tiktok
Region: Asia
  Interpretation: Weak performance relative to region/platform norms.

Video 2:
  Predicted Category: likely
  Trending Score: 0.25/100
Platform: youtube
Region: Europe
  Interpretation: Weak performance relative to region/platform norms.

Video 3:
  Predicted Category: likely
  Trending Score: 3.50/100
Platform: tiktok
Region: Americas
  Interpretation: Weak performance relative to region/platform norms.

Video 4:
  Predicted Category: low
  Trending Score: 0.25/100
Platform: youtube
Region: Africa
  Interpretation: Weak performance relative to region/platform norms.

Video 5:
  Predicted Category: likely
  Trending Score: 1.75/100
Platform: tiktok
Region: MENA
  Interpretation: Weak performance relative to region/platform norms.

Video 6:
  Predicted Category: low
  Trending Score: 3.75/100
Platform: youtube
Region: Oceania
  Interpretation: Weak performance relative to region/platform norm